In [25]:
#Appending new rows to DataFrame
import pandas as pd
import numpy as np
names=pd.read_csv('data/names.csv')
new_data_list=['Aria',1]
names.loc[4]=new_data_list #加一个record
names.loc[len(names)]={'Name':'Zayd','Age':1}
names.loc[len(names)]=pd.Series({'Name':'Peter','Age':2})
names

name1=pd.read_csv('data/names.csv')
name1.append({'Name':'Peter','Age':2},ignore_index=True) 
#ignore_index=True的时候，旧的会被removed and replaced by a rangeindex from o to n-1

ball_16=pd.read_csv('data/baseball16.csv')
data_dict=ball_16.iloc[0].to_dict()
#clear out old values with a dictionary comprehension assgind any previous 
#string value as an empty string and all others, missing values

new_data_dict={k: ''if isinstance(v,str) else
              np.nan for k, v in data_dict.items()}

random_data=[]
for i in range(10):
    d=dict()
    for k,v in data_dict.items():
        if isinstance(v,str):
            d[k]=np.random.choice(list('abcdef'))
        else:
            d[k]=np.random.randint(10)
    random_data.append(pd.Series(d,name=i+len(ball_16)))
    
random_data[0]

playerID    a
yearID      5
stint       7
teamID      d
lgID        f
G           4
AB          3
R           3
H           8
2B          3
3B          2
HR          4
RBI         5
SB          5
CS          8
BB          3
SO          3
IBB         7
HBP         0
SH          1
SF          0
GIDP        5
Name: 16, dtype: object

In [36]:
#Concat multiple DataFrame together
stock2016=pd.read_csv('data/stocks_2016.csv',index_col='Symbol')
stock2017=pd.read_csv('data/stocks_2017.csv',index_col='Symbol')
#pd.concat([stock2016,stock2017],axis=0)

#pd.concat([stock2016,stock2017],keys=['2016','2017'],names=['Year','Symbol'],axis=0)
pd.concat([stock2016,stock2017],join='inner',keys=['2016','2017'],names=['Year','Symbol'],axis=1)



Year     2016            2017          
Symbol Shares Low High Shares  Low High
Symbol                                 
AAPL       80  95  110     50  120  140
TSLA       50  80  130    100  100  300

In [52]:
#understanding the difference between concat, join and merge
import pandas as pd
from IPython.display import display_html
years=2016,2017,2018
stock_tables=[pd.read_csv('data/stocks_{}.csv'.format(year),index_col='Symbol') for year in years]

def display_frames(frames,num_spaces=0):
    t_style='<table style="display:inline;"'
    tables_html=[df.to_html().replace('<table',t_style) for df in frames]
    
    space='&nbsp;'*num_spaces
    display_html(space.join(tables_html),raw=True)
    
display_frames(stock_tables,30)

pd.concat(stock_tables,keys=[2016,2017,2018])

#zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表
pd.concat(dict(zip(years,stock_tables)),axis=1,sort=False)


#join
stocks_2016,stocks_2017,stocks_2018=stock_tables
stocks_2016.join(stocks_2017,lsuffix='_2016', rsuffix='_2017',how='outer')

other=[stocks_2017.add_suffix('_2017'),
      stocks_2018.add_suffix('_2018')]
stocks_2016.add_suffix('_2016').join(other,how='outer',sort=True)

#merge
stocks_2016.merge(stocks_2017,left_index=True,right_index=True)

step1=stocks_2016.merge(stocks_2017,left_index=True,right_index=True,how='outer',suffixes=('_2016','_2017'))
stock_merge=step1.merge(stocks_2018.add_suffix('_2018'),left_index=True,right_index=True,how='outer')
stock_merge

#let us turn on comparation to dataset where we are interested in alining the values of columns 
#and not the index or column labels
names=['prices','transactions']
food_tables=[pd.read_csv('data/food_{}.csv'.format(x)) for x in names]

food_prices,food_transactions=food_tables
display_frames(food_tables,30)
food_transactions.merge(food_prices.query('Date==2017'),on=['item','store'],how='left')

#Use join do the same
food_prices_join=food_prices.query('Date==2017').set_index(['item','store'])
food_transactions.join(food_prices_join,on=['item','store'])

#use concat
#pd.concat([food_transactions.set_index(['item','store'])],
#          food_prices.set_index(['item','store'])],
#          axis='columns'
#in this particular case, an error would be produced as a duplicate index values occuses in at lease 
#on of the DF (with the item steak in store b)

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70
,Shares,Low,High
Symbol,,,
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85


/Users/Lampard/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6359: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  verify_integrity=True)


,item,store,price,Date
0,pear,A,0.99,2017
1,pear,B,1.99,2017
2,peach,A,2.99,2017
3,peach,B,3.49,2017
4,banana,A,0.39,2017
5,banana,B,0.49,2017
6,steak,A,5.99,2017
7,steak,B,6.99,2017
8,steak,B,4.99,2015
,custid,item,store,quantity


,custid,item,store,quantity,price,Date
0,1,pear,A,5,0.99,2017.0
1,1,banana,A,10,0.39,2017.0
2,2,steak,B,3,6.99,2017.0
3,2,pear,B,1,1.99,2017.0
4,2,peach,B,2,3.49,2017.0
5,2,steak,B,1,6.99,2017.0
6,2,coconut,B,4,NaN,NaN


In [57]:
#read all csv file in a folder
import glob
import pandas as pd
df_list=[]
for filename in glob.glob('data/gas prices/*.csv'):
    df_list.append(pd.read_csv(filename,index_col='Week',parse_dates=['Week']))
    
gas=pd.concat(df_list,axis='columns')
gas.head()

,Midgrade,Premium,Diesel,All Grades,Regular
Week,,,,,
2017-09-25,2.859,3.105,2.788,2.701,2.583
2017-09-18,2.906,3.151,2.791,2.750,2.634
2017-09-11,2.953,3.197,2.802,2.800,2.685
2017-09-04,2.946,3.191,2.758,2.794,2.679
2017-08-28,2.668,2.901,2.605,2.513,2.399


In [60]:
#Connect to SQL databases
import pandas as pd
from sqlalchemy import create_engine
engine=create_engine('sqlite:///data/chinook.db')
tracks=pd.read_sql_table('tracks',engine)
tracks.head()
genres=pd.read_sql_table('genres',engine)
genres.head()

genre_track=genres.merge(tracks[['GenreId','Milliseconds']],on=['GenreId'],how='left').drop('GenreId',axis=1)
genre_time=genre_track.groupby('Name')['Milliseconds'].mean()
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_timedelta.html
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.dt.floor.html
pd.to_timedelta(genre_time,unit='ms').dt.floor('s').sort_values(ascending=False)

cust=pd.read_sql_table('customers',engine,columns=['CustomerId','FirstName','LastName'])
invoice=pd.read_sql_table('invoices',engine,columns=['InvoiceId','CustomerId'])
ii=pd.read_sql_table('invoice_items',engine,columns=['InvoiceId','UnitPrice','Quantity'])
cust_inv=cust.merge(invoice,on='CustomerId').merge(ii,on='InvoiceId')
total=cust_inv['Quantity']*cust_inv['UnitPrice']
#assign means add a new columns
#cust_inv['Total']=cust_inv['Quantity']*cust_inv['UnitPrice']
cust_inv.assign(Total=total).groupby(['CustomerId','FirstName','LastName'])['Total'].sum()

CustomerId  FirstName  LastName    
1           Luís       Gonçalves       39.62
2           Leonie     Köhler          37.62
3           François   Tremblay        39.62
4           Bjørn      Hansen          39.62
5           František  Wichterlová     40.62
6           Helena     Holý            49.62
7           Astrid     Gruber          42.62
8           Daan       Peeters         37.62
9           Kara       Nielsen         37.62
10          Eduardo    Martins         37.62
11          Alexandre  Rocha           37.62
12          Roberto    Almeida         37.62
13          Fernanda   Ramos           37.62
14          Mark       Philips         37.62
15          Jennifer   Peterson        38.62
16          Frank      Harris          37.62
17          Jack       Smith           39.62
18          Michelle   Brooks          37.62
19          Tim        Goyer           38.62
20          Dan        Miller          39.62
21          Kathy      Chase           37.62
22          Heather

In [66]:
#Write SQL
#you can write SQL query as string and pass it to the read_sql_query fucntion
sql_string1='''
select
    Name,
    time(avg(Milliseconds)/1000, 'unixepoch') as avg_time
from
    (
    select
    g.Name,
    t.Milliseconds
    from
    genres as g
    join
    tracks as t
    on
    g.genreid==t.genreid
    )
group by
    Name
order by
    avg_time
'''
pd.read_sql_query(sql_string1,engine) 

,Name,avg_time
0,Rock And Roll,00:02:14
1,Opera,00:02:54
2,Hip Hop/Rap,00:02:58
3,Easy Listening,00:03:09
4,Bossa Nova,00:03:39
5,R&B/Soul,00:03:40
6,World,00:03:44
7,Pop,00:03:49
8,Latin,00:03:52
9,Alternative & Punk,00:03:54
